from __future__ import division

### pyvisa

Probably won't use this library because it doesn't work on ubuntu out of the box

In [ ]:
import visa
rm = visa.ResourceManager()

### pyusb

In [7]:
import usb.core

In [8]:
# usb.core.show_devices(verbose=True)
usb.core.show_devices()

DEVICE ID 8087:8000 on Bus 004 Address 002, Hub
DEVICE ID 1d6b:0002 on Bus 004 Address 001, Hub
DEVICE ID 8087:8008 on Bus 003 Address 002, Hub
DEVICE ID 1d6b:0002 on Bus 003 Address 001, Hub
DEVICE ID 1d6b:0003 on Bus 002 Address 001, Hub
DEVICE ID 13d3:3394 on Bus 001 Address 003, Wireless Controller
DEVICE ID 1ab1:0588 on Bus 001 Address 005, Specified at interface
DEVICE ID 1d6b:0002 on Bus 001 Address 001, Hub

In [9]:
for dev in usb.core.find(find_all=True):
#     print(dev)
#     print(dir(dev))
#     print(dev.idVendor)
#     print(dev.idProduct)
    if dev.idVendor == 6833:
        print(dev)

DEVICE ID 1ab1:0588 on Bus 001 Address 005 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x110 USB 1.1
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :   0x40 (64 bytes)
 idVendor               : 0x1ab1
 idProduct              : 0x0588
 bcdDevice              :  0x100 Device 1.0
 iManufacturer          :    0x1 Rigol Technologies         
 iProduct               :    0x2 DS1000 SERIES                   
 iSerialNumber          :    0x3 DS1EB142004074 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 2 mA ====================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x27 (39 bytes)
   bNumInterfaces       :    0x1
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0xc0 S

In [12]:
dev = usb.core.find(idVendor=0x1ab1, idProduct=0x0588)
if dev.is_kernel_driver_active(0):
    reattach = True
    dev.detach_kernel_driver(0)
dev.set_configuration()

USBError: [Errno 16] Resource busy

In [11]:
cfg = dev.get_active_configuration()
intf = cfg[(0,0)]

In [33]:
cfg[(0,0)]

<INTERFACE 0: Application Specific>

In [13]:
# print(dir(intf))
intf.endpoints()

(<ENDPOINT 0x1: Bulk OUT>,
 <ENDPOINT 0x82: Bulk IN>,
 <ENDPOINT 0x83: Interrupt IN>)

In [35]:
ep = usb.util.find_descriptor(intf, 
    custom_match = lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_OUT)
print(ep)

      ENDPOINT 0x1: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x1 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :   0x40 (64 bytes)
       bInterval        :    0x0


In [45]:
dev.write(0x82, "*IDN?", timeout=10000)

USBError: [Errno 110] Operation timed out

failure

### usbtmc

In [1]:
import usbtmc

In [7]:
devices = usbtmc.list_devices()

In [8]:
for dev in devices:
    print(dev.manufacturer)
    print(dev.idProduct)
    print(dev.idVendor)

USBError: [Errno 13] Access denied (insufficient permissions)

In [18]:
d = usbtmc.list_devices()[0]
print(dir(d))
print(d.manufacturer)

['_Device__default_timeout', '_Device__get_def_tmo', '_Device__get_timeout', '_Device__set_def_tmo', '__class__', '__del__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__getitem__', '__hash__', '__init__', '__iter__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_ctx', '_get_full_descriptor_str', '_manufacturer', '_product', '_serial_number', '_str', 'address', 'attach_kernel_driver', 'bDescriptorType', 'bDeviceClass', 'bDeviceProtocol', 'bDeviceSubClass', 'bLength', 'bMaxPacketSize0', 'bNumConfigurations', 'bcdDevice', 'bcdUSB', 'bus', 'clear_halt', 'configurations', 'ctrl_transfer', 'default_timeout', 'detach_kernel_driver', 'get_active_configuration', 'iManufacturer', 'iProduct', 'iSerialNumber', 'idProduct', 'idVendor', 'is_kernel_driver_active', 'manufacturer', 'port_number', 'product', 'read', 'reset', 'serial_number', 'set_configuration', 'set_interfac

In [3]:
instr = usbtmc.Instrument(0x1ab1, 0x0588)

In [4]:
instr.device

<DEVICE ID 1ab1:0588 on Bus 001 Address 005>

In [5]:
instr.ask("*IDN?")

u'Rigol Technologies,DS1102E,DS1EB142004074,00.03.01.00.00'

##### Finally a success